In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Widgets

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("catalog","catalog_ecommerce")
dbutils.widgets.text("schema_source","silver")
dbutils.widgets.text("shema_sink","gold")

#Constantes

In [0]:
catalog = dbutils.widgets.get("catalog")
schema_source = dbutils.widgets.get("schema_source")
schema_sink = dbutils.widgets.get("shema_sink")

#Metricas

In [0]:
df_delivery = spark.sql(f"""
    SELECT
        c.customer_state as Location,

        -- metricas de entrega
        COUNT(DISTINCT o.order_id) as total_deliveries,
        ROUND(AVG(o.delivery_days), 1) as avg_delivery_days,
        MIN(o.delivery_days) as min_delivery_days,
        MAX(o.delivery_days) as max_delivery_days,

        --
        ROUND(AVG(o.delivery_vs_estimated_time)) as delivery_vs_estimated_time,
        SUM(CASE WHEN o.Is_late THEN 1 ELSE 0 END) as late_deliveries,
        SUM(CASE WHEN o.delivery_vs_estimated_time <= 0 THEN 1 ELSE 0 END) as on_time_deliveries


    FROM {catalog}.{schema_source}.orders as o
    INNER JOIN {catalog}.{schema_source}.customers as c
        on c.customer_id = o.customer_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_state
    ORDER BY total_deliveries DESC
""")

In [0]:
df_delivery.write.mode("overwrite").saveAsTable(f"{catalog}.{schema_sink}.Delivery_by_states")